<a href="https://colab.research.google.com/github/DanielWarfield1/MLWritingAndResearch/blob/main/GroundX_Llama_3_1_70B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GroundX + Llama 3.1 70B

In [ ]:
#for calling generative model
!pip install unifyai

#for performing advanced RAG
!pip install groundx-python-sdk

In [ ]:
"""Authenticating GroundX for advanced RAG and Unify
"""

from google.colab import userdata
import os
unify_api_key = userdata.get('xxxxxxxxxxxx')
os.environ['UNIFY_KEY'] = unify_api_key

from groundx import Groundx
groundx = Groundx(
    api_key=userdata.get('xxxxxxxxxx'),
)

In [ ]:
# Specifying Bucket
# Follow this tutorial to upload content to your own bucket:
# https://www.groundx.ai/post/groundx-in-5-minutes-building-a-legal-assistant
bucketId = 7802

In [ ]:
def gx_retreive(query):
    """Getting GroundX context from docs based on query
    """
    #getting GroundX Retreival
    response = groundx.search.content(
        id=bucketId,
        query=query
    )

    #Getting GroundX Recommended Context
    return response.body['search']['text']

In [ ]:
def gx_retreive_and_augment(query):
    """Constructing an augmented prompt
    """

    #getting context
    context = gx_retreive(query)[:20000]

    #defining a high level prompt so the LLM knows what to do
    system_prompt = 'you are a helpful AI agent tasked with helping\
    users to extract information from the context below'

    #based on OpenAI's new formatting
    augmented_prompt = '\n'.join([system_prompt+'\n\n===\n'+context+'\n===','query:',query])

    return augmented_prompt

In [ ]:
from unify import Unify

#defining model and endpoint
unify = Unify("llama-3.1-70b-chat@together-ai")

#generating based on GroundX Augmented Prompt
response = unify.generate(gx_retreive_and_augment('What is the branch rate in Belgium?'))
response

'According to the first text excerpt, the "Branch Tax Rate (%)" in Belgium is a flat 25%.'